In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_july=pd.read_csv("data/train_July.csv")
train_aug=pd.read_csv("data/train_Aug.csv")
weather=pd.read_csv("data/weather.csv")
poi=pd.read_csv("data/poi.csv")
test=pd.read_csv("data/test_id_Aug_agg_public5k.csv")

In [3]:
def evaluate(X_data):
    return np.mean(abs(X_data['count']-X_data['pre']))
mon=[3,10,17,24,31]
tue=[4,11,18,25]
wed=[5,12,19,26]
thr=[6,13,20,27]
fri=[7,14,21,28]
sat=[1,8,15,22,29]
sun=[2,9,16,23,30]
def july_cal_day(x):
    if x in mon:
        return 1
    elif x in tue:
        return 2
    elif x in wed:
        return 3
    elif x in thr:
        return 4
    elif x in fri:
        return 5
    elif x in sat:
        return 6
    elif x in sun:
        return 7
def aug_cal_day(x):
    if x!=7:
        return x%7
    else:
        return 7

In [4]:
train_july['day']=train_july['create_date'].apply(lambda x:int(str(x).split('-')[2]))
train_aug['day']=train_aug['create_date'].apply(lambda x:int(str(x).split('-')[2]))
test['day']=test['create_date'].apply(lambda x:int(str(x).split('-')[2]))

In [5]:
train_july['day_week']=train_july['day'].apply(july_cal_day)
train_aug['day_week']=train_aug['day'].apply(aug_cal_day)
test['day_week']=test['day'].apply(aug_cal_day)

In [6]:
train=pd.concat([train_july,train_aug])

In [7]:
train_dict=train.groupby(['start_geo_id','end_geo_id','create_date','create_hour'])['id'].count().to_dict()
train['count']=train.apply(lambda x:train_dict[tuple([x['start_geo_id'],x['end_geo_id'],x['create_date'],x['create_hour']])],axis=1)

In [8]:
train=train[train[['start_geo_id','end_geo_id','create_date','create_hour']].duplicated()==False]
train=train[['start_geo_id','end_geo_id','create_date','create_hour','day_week','count']]

In [9]:
test.drop(['day','create_date'],axis=1,inplace=True)
train.drop('create_date',axis=1,inplace=True)

In [10]:
train.head()

,start_geo_id,end_geo_id,create_hour,day_week,count
0,6d7827e8dcfa09497954a31e6f7e6ee6,85e49ded1fa70a7bfa01ab0212a6e538,0,6,3
1,27d75f17e61587172fe7a6827bbaa198,f5dc996f6aa097f7a84a9bcfe58ed55c,0,6,3
2,f92dfcc31699ad56d967a57673b8fc65,8c269e40d177f46840aff30baeb25e29,0,6,5
3,92e1e8020813ef939183e345626b442a,f80c4ceeb36264b42e34d6c4c2cb9b4c,0,6,3
4,ca902ab284d4bb9e8c2a0bb58d55cf7b,6da3e22032d543214b2893d1c9f2b9e1,2,6,3


In [11]:
merged_data=test.merge(train,on=['start_geo_id','end_geo_id','day_week'],how='inner')

In [17]:
result=dict()
def get_count(x):
    if x['test_id'] in result.keys():
        return
    value=np.array([-1,-1])
    data=merged_data[merged_data['test_id']==x['test_id']]
    idx=data['create_hour_x']>data['create_hour_y']
    wt=1.0/np.exp(abs(data['create_hour_x']-data['create_hour_y']))
    if sum(idx)>0:
        value[0]=sum(data['count'][idx]*wt[idx])/sum(wt[idx])
    if sum(idx==False)>0:
        value[1]=sum(data['count'][idx==False]*wt[idx==False])/sum(wt[idx==False])
    w=np.array([0.5,0.5])
    flag=value>0
    pre=sum(value[flag]*w[flag])/sum(w[flag])
    result[x['test_id']]=pre

In [18]:
merged_data.apply(get_count,axis=1)

0         None
1         None
2         None
3         None
4         None
5         None
6         None
7         None
8         None
9         None
10        None
11        None
12        None
13        None
14        None
15        None
16        None
17        None
18        None
19        None
20        None
21        None
22        None
23        None
24        None
25        None
26        None
27        None
28        None
29        None
          ... 
135269    None
135270    None
135271    None
135272    None
135273    None
135274    None
135275    None
135276    None
135277    None
135278    None
135279    None
135280    None
135281    None
135282    None
135283    None
135284    None
135285    None
135286    None
135287    None
135288    None
135289    None
135290    None
135291    None
135292    None
135293    None
135294    None
135295    None
135296    None
135297    None
135298    None
Length: 135299, dtype: object

In [19]:
import winsound
for i in range(4):
    winsound.Beep(600,1000)

In [20]:
result

{0L: 15.5,
 1L: 9.0,
 2L: 6.5,
 3L: 8.0,
 4L: 5.5,
 5L: 1.0,
 6L: 4.5,
 7L: 2.0,
 8L: 3.5,
 9L: 2.0,
 10L: 2.5,
 11L: 1.0,
 12L: 2.5,
 13L: 4.5,
 14L: 2.5,
 15L: 1.0,
 16L: 6.5,
 17L: 1.0,
 18L: 3.5,
 19L: 4.0,
 20L: 4.0,
 21L: 6.0,
 22L: 2.0,
 23L: 1.5,
 24L: 7.0,
 25L: 3.0,
 26L: 1.0,
 27L: 3.0,
 28L: 4.5,
 29L: 3.0,
 30L: 4.0,
 31L: 1.0,
 32L: 3.0,
 33L: 3.5,
 34L: 1.5,
 35L: 1.5,
 36L: 4.0,
 37L: 8.0,
 38L: 3.5,
 39L: 6.5,
 40L: 3.5,
 41L: 1.0,
 42L: 1.0,
 43L: 2.0,
 44L: 3.5,
 45L: 9.5,
 46L: 3.0,
 47L: 1.0,
 48L: 3.0,
 49L: 6.5,
 50L: 6.0,
 51L: 2.5,
 52L: 2.5,
 53L: 1.5,
 54L: 1.0,
 55L: 3.5,
 56L: 5.0,
 57L: 2.0,
 58L: 3.0,
 59L: 2.5,
 60L: 3.5,
 61L: 5.0,
 62L: 2.0,
 63L: 1.0,
 64L: 1.0,
 65L: 1.0,
 66L: 6.0,
 67L: 4.0,
 68L: 1.0,
 69L: 1.0,
 70L: 1.0,
 71L: 5.5,
 72L: 2.0,
 73L: 5.0,
 74L: 1.5,
 75L: 4.5,
 76L: 1.0,
 77L: 1.0,
 78L: 1.0,
 79L: 7.5,
 81L: 2.0,
 82L: 1.0,
 83L: 1.5,
 84L: 1.0,
 85L: 3.0,
 86L: 1.0,
 87L: 1.5,
 88L: 1.0,
 89L: 2.0,
 90L: 1.0,
 91L: 1.0,
 93L: 1.

In [21]:

sub=pd.DataFrame({"test_id":test['test_id']})
sub['count']=test['test_id'].apply(lambda x:result[x] if x in result.keys() else 1)
sub.to_csv("../result/submission.csv",index=False)

In [22]:
len(result.keys())

4423